### Rastreo de noticias en Google News sobre temas de la Ciudad de México que competan o sean de interés del Instituto de Planeación de la Ciudad de México

El script extrae las notas de Google News de los temas de agua, movilidad, planeación, salud y seguridad de la Ciudad de México. El script extrae los títulos, url y fecha de las notas.
Además se ordena y extrae en el formato de entrega diaria para el Instituto de Planeación.

In [48]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
from docx import Document
from docx.shared import RGBColor
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from docx.shared import Pt
import locale
import os


Extraer notas a partir de los temas relevantes y palabras clave para localizarlos en Ciudad de México

In [49]:
# Temas para la búsqueda
temas = [
    "planeación", "infraestructura", "agua", "pgd", "pgot", "metropolitano",
    "consulta%20pública", "consulta%20indígena", "vivienda",
    "ordenamiento%20territorial", "movilidad", 
    "salud", "gestión%20de%20riesgos", "seguridad%20ciudadana", 
    "participación%20ciudadana", "medio%20ambiente", "cultura", 
    "asentamientos%20irregulares", "ipdp", "congreso%20de%20la%20ciudad%20de%20méxico",
    "transporte%20público", "espacio%20público",
    "patricia%20ramírez%20kuri", "turismo", "educación", "economía", "empleo" 'transparencia '  
]

In [50]:
def limpiar_tema(t):
    t = t.replace('%20', ' ')
    palabras = t.split()
    excepciones = ['de', 'la', 'del', 'y', 'en', 'a']  # palabras que no se capitalizan
    palabras_limpias = []

    for i, palabra in enumerate(palabras):
        if i == 0 or palabra.lower() not in excepciones:
            palabras_limpias.append(palabra.capitalize())
        else:
            palabras_limpias.append(palabra.lower())
    
    return ' '.join(palabras_limpias)


In [51]:
#Palabras clave para filtrar solo noticias de la Ciudad de México
palabras_clave_cdmx = [
    "Ciudad de México", "CDMX", "Iztapalapa", "Coyoacán", "Cuauhtémoc", "Benito Juárez", 
    "Miguel Hidalgo", "Xochimilco", "Tlalpan", "Gustavo A. Madero", "Venustiano Carranza",
    "Azcapotzalco", "Tláhuac", "Álvaro Obregón", "Milpa Alta", "Magdalena Contreras"
]

In [52]:
#Lista de medios relevantes
medios = [
    'El Universal', 'Reforma', 'MVS Noticias', 'SinEmbargo MX', 'Reporte Indigo',
    'Gobierno de la Ciudad de México', 'La Silla Rota', 'MSN', 
    'Gobierno de México', 'Animal Político', 'La Razón de México',
    'Excélsior', 'Radio Fórmula', 'N+', 'Milenio',
    'SSC-CdMx', 'Infobae México', 'TV Azteca', 'Quadratín México', 'ContraRéplica',
    'La Jornada', 'Uno TV Noticias', 'Ovaciones', 'ADN 40', 'El Heraldo de México',
    'El Economista', 'El País', 'Radio Fórmula'
]

#Convertir la lista de medios en una expresión regular
medios_regex = '|'.join(map(re.escape, medios))


In [53]:
#Función para extraer notas
def get_notas(tema, periodo):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36'}
    
    url = f'https://news.google.com/search?q={tema}%20cdmx%20when%3A{periodo}d&hl=es-419&gl=MX&ceid=MX%3Aes-419'
    
    print(url)

    response = requests.get(url, headers=headers)
    sopa = BeautifulSoup(response.text, 'html.parser')
    resultados = sopa.find_all('article')
    
    #Listas para almacenar datos
    titulos, urls, fechas, medios_lista = [], [], [], []

    for resultado in resultados:
        #Extraer título
        titulo_elem = resultado.find('a', class_='JtKRv')
        titulo = titulo_elem.text.strip() if titulo_elem else None

        #Extraer URL
        url_elem = resultado.find('a')
        url = 'https://news.google.com' + url_elem.get('href')[1:] if url_elem and url_elem.get('href') else None

        #Extraer fecha
        fecha_elem = resultado.find('time')
        fecha = fecha_elem['datetime'] if fecha_elem else None

        #Extraer medio
        medio_elem = resultado.find('div', class_='vr1PYe')
        medio = medio_elem.text.strip() if medio_elem else None

        #Guardar en listas
        titulos.append(titulo)
        urls.append(url)
        fechas.append(fecha)
        medios_lista.append(medio)

    #Crear DataFrame
    df = pd.DataFrame({
        'titulo': titulos,
        'url': urls,
        'fecha_nota': fechas,
        'medio': medios_lista,
        'fecha_consulta': pd.to_datetime('today').strftime('%Y-%m-%d'),
        'tema': tema
    })

    return df

In [54]:
# Definir el periodo de búsqueda
periodo = 2

# Extraer notas de todos los temas
consolidado_notas = pd.concat([get_notas(tema, periodo) for tema in temas], ignore_index=True)

# Filtrar por palabras clave de CDMX
consolidado_cdmx = consolidado_notas[consolidado_notas['titulo'].fillna('').str.contains('|'.join(palabras_clave_cdmx), regex=True, case=False)]

https://news.google.com/search?q=planeación%20cdmx%20when%3A2d&hl=es-419&gl=MX&ceid=MX%3Aes-419
https://news.google.com/search?q=infraestructura%20cdmx%20when%3A2d&hl=es-419&gl=MX&ceid=MX%3Aes-419
https://news.google.com/search?q=agua%20cdmx%20when%3A2d&hl=es-419&gl=MX&ceid=MX%3Aes-419
https://news.google.com/search?q=pgd%20cdmx%20when%3A2d&hl=es-419&gl=MX&ceid=MX%3Aes-419
https://news.google.com/search?q=pgot%20cdmx%20when%3A2d&hl=es-419&gl=MX&ceid=MX%3Aes-419
https://news.google.com/search?q=metropolitano%20cdmx%20when%3A2d&hl=es-419&gl=MX&ceid=MX%3Aes-419
https://news.google.com/search?q=consulta%20pública%20cdmx%20when%3A2d&hl=es-419&gl=MX&ceid=MX%3Aes-419
https://news.google.com/search?q=consulta%20indígena%20cdmx%20when%3A2d&hl=es-419&gl=MX&ceid=MX%3Aes-419
https://news.google.com/search?q=vivienda%20cdmx%20when%3A2d&hl=es-419&gl=MX&ceid=MX%3Aes-419
https://news.google.com/search?q=ordenamiento%20territorial%20cdmx%20when%3A2d&hl=es-419&gl=MX&ceid=MX%3Aes-419
https://news.google.

In [55]:
print(f'Total de notas encontradas: {len(consolidado_notas)}')
print(f'Notas filtradas por palabras clave CDMX: {len(consolidado_cdmx)}')


Total de notas encontradas: 884
Notas filtradas por palabras clave CDMX: 707


In [56]:
# Formatear fechas
consolidado_cdmx['fecha_nota'] = pd.to_datetime(consolidado_cdmx['fecha_nota'], errors='coerce')

# Filtrar solo medios relevantes
consolidado_relevantes = consolidado_cdmx[consolidado_cdmx['medio'].fillna('').str.contains(medios_regex, regex=True, case=False)]


/var/folders/wv/qn33ppwj6078glchqvgyx29m0000gn/T/ipykernel_912/908425338.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consolidado_cdmx['fecha_nota'] = pd.to_datetime(consolidado_cdmx['fecha_nota'], errors='coerce')


In [57]:
# Obtener top 3 por medio prioritario
#def obtener_top_3_por_medio(df):
   # df = df.sort_values(by=['medio', 'fecha_nota'], ascending=[True, False])
   # return df.groupby('medio').head(3)

In [58]:
#consolidado_top3 = obtener_top_3_por_medio(consolidado_relevantes)


In [59]:
# Asegurarnos de que la columna de fecha no tenga zona horaria
consolidado_top3['fecha_nota'] = consolidado_top3['fecha_nota'].dt.tz_localize(None)

# Guardar en Excel
consolidado_top3.to_excel("noticias_cdmx_prioritarias.xlsx", index=False)

In [60]:

print(f'Total de notas después de seleccionar las 3 más relevantes por medio: {len(consolidado_top3)}')
print("Archivo guardado como 'noticias_cdmx_prioritarias.xlsx'")

Total de notas después de seleccionar las 3 más relevantes por medio: 61
Archivo guardado como 'noticias_cdmx_prioritarias.xlsx'


Almacenar las notas en el formato de entrega

In [61]:
# Función para agregar hipervínculos con fuente Arial
def add_hyperlink(paragraph, text, url):
    part = paragraph._parent.part
    r_id = part.relate_to(url, "http://schemas.openxmlformats.org/officeDocument/2006/relationships/hyperlink", is_external=True)

    hyperlink = OxmlElement("w:hyperlink")
    hyperlink.set(qn("r:id"), r_id)

    run = OxmlElement("w:r")
    rPr = OxmlElement("w:rPr")

    # Fuente Arial
    rFonts = OxmlElement("w:rFonts")
    rFonts.set(qn("w:ascii"), "Arial")
    rFonts.set(qn("w:hAnsi"), "Arial")
    rFonts.set(qn("w:eastAsia"), "Arial")
    rPr.append(rFonts)

    # Color azul
    color = OxmlElement("w:color")
    color.set(qn("w:val"), "0000FF")
    rPr.append(color)

    # Subrayado
    underline = OxmlElement("w:u")
    underline.set(qn("w:val"), "single")
    rPr.append(underline)

    run.append(rPr)

    text_element = OxmlElement("w:t")
    text_element.text = text
    run.append(text_element)

    hyperlink.append(run)
    paragraph._element.append(hyperlink)


In [62]:
# Función de respaldo si locale falla
def fecha_en_espanol(fecha):
    meses = {
        'January': 'enero', 'February': 'febrero', 'March': 'marzo', 'April': 'abril',
        'May': 'mayo', 'June': 'junio', 'July': 'julio', 'August': 'agosto',
        'September': 'septiembre', 'October': 'octubre', 'November': 'noviembre', 'December': 'diciembre'
    }
    fecha_str = fecha.strftime('%d de %B de %Y')
    for en, es in meses.items():
        fecha_str = fecha_str.replace(en, es)
    return fecha_str


In [63]:
#Cargar la tabla de excel
df = pd.read_excel("noticias_cdmx_prioritarias.xlsx")
df['medio'] = df['medio'].fillna('Fuente desconocida')
df['url'] = df['url'].fillna('#')
df['fecha_nota'] = pd.to_datetime(df['fecha_nota'], errors='coerce').dt.strftime('%Y-%m-%d')

def limpiar_tema(t):
    return str(t).strip().capitalize()

df['tema'] = df['tema'].apply(limpiar_tema)
df['tema'] = df['tema'].str.replace('%20', ' ', regex=False)

df_grouped = df.groupby(['fecha_nota', 'medio', 'titulo', 'url']).agg({'tema': lambda x: ', '.join(set(x))}).reset_index()

In [64]:
#Crear un documento de Word
doc = Document()


In [65]:
#Homologar la fuente a Arial
style = doc.styles['Normal']
font = style.font
font.name = 'Arial'
font.size = Pt(10)

style.element.rPr.rFonts.set(qn('w:eastAsia'), 'Arial')

#Ponerlo en español utf-8
try:
    locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
except:
    try:
        locale.setlocale(locale.LC_TIME, 'Spanish_Spain.1252')
    except:
        print('No se pudo establecer la localización en español. Usando traducción manual')

fecha_actual = pd.to_datetime('today')
try:
    fecha_formateada = fecha_actual.strftime('%d de %B de %Y')
except:
    fecha_formateada = fecha_en_espanol(fecha_actual)


In [66]:
#Formato de entrega
 
# Título
titulo = doc.add_heading("Monitoreo de Medios y Redes Sociales", level=1)
titulo.alignment = 1  

for run in titulo.runs:
    run.font.name = 'Arial'
    run.font.size = Pt(14)

# Subtítulo
subtitulo1 = doc.add_paragraph("Temas prioritarios de planeación de la Ciudad de México")
subtitulo1.alignment = 1  

for run in subtitulo1.runs:
    run.font.name = 'Arial'
    run.font.size = Pt(12)  

subtitulo2 = doc.add_paragraph(f"Informe con corte al día {fecha_formateada}")
subtitulo2.alignment = 1 

for run in subtitulo2.runs:
    run.font.name = 'Arial'
    run.font.size = Pt(12)  

#Crear tabla
table = doc.add_table(rows=1, cols=4)
table.style = 'Table Grid'

#Variables de la tabla
headers = ["Fecha de la nota", "Fuente", "Idea central", "Temas"]
for i, texto in enumerate(headers):
    p = table.rows[0].cells[i].paragraphs[0]
    run = p.add_run(texto)
    run.font.name = 'Arial'
    run.font.size = Pt(10)
    run._element.rPr.rFonts.set(qn('w:eastAsia'), 'Arial')

for _, row in df_grouped.iterrows():
    row_cells = table.add_row().cells

    #Fecha
    p_fecha = row_cells[0].paragraphs[0]
    run_fecha = p_fecha.add_run(row["fecha_nota"] if pd.notna(row["fecha_nota"]) else "No encontrado")
    run_fecha.font.name = 'Arial'
    run_fecha.font.size = Pt(10)
    run_fecha._element.rPr.rFonts.set(qn('w:eastAsia'), 'Arial')

    #Agregar hipervínculo
    p_medio = row_cells[1].paragraphs[0]
    add_hyperlink(p_medio, row["medio"], row["url"])

    #Título
    p_titulo = row_cells[2].paragraphs[0]
    run_titulo = p_titulo.add_run(row["titulo"])
    run_titulo.font.name = 'Arial'
    run_titulo.font.size = Pt(10)
    run_titulo._element.rPr.rFonts.set(qn('w:eastAsia'), 'Arial')

    #Tema
    p_tema = row_cells[3].paragraphs[0]
    run_tema = p_tema.add_run(row["tema"])
    run_tema.font.name = 'Arial'
    run_tema.font.size = Pt(10)
    run_tema._element.rPr.rFonts.set(qn('w:eastAsia'), 'Arial')


In [67]:
# Guardar
doc.save(f"Monitoreo_Medios_Redes_ {fecha_formateada}.docx")